In [14]:
import json
import time
import random
from pygame import mixer

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    UnexpectedAlertPresentException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

options = Options()
# options.headless = True
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)

[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:02<00:00, 3.14MB/s]


In [15]:
class Instagram:
    def __init__(self) -> None:
        file = open("./credentials.json")
        credentials = json.load(file)
        self.username = credentials["username"]
        self.password = credentials["password"]
        self.ranges = [
            range(60, 90),
            range(120, 150),
            range(210, 240),
            range(240, 270),
            range(330, 360),
            range(390, 420),
            range(480, 510),
        ]
        self.posts_liked = 0
        self.pauses = []

    def login(self) -> None:
        try:
            driver.get("https://www.instagram.com/accounts/login/")
            driver.implicitly_wait(3)
            username_input = driver.find_element(By.NAME, "username")
            username_input.send_keys(self.username)
            password_input = driver.find_element(By.NAME, "password")
            password_input.send_keys(self.password)
            button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
            button.click()
        except NoSuchElementException:
            print("Already logged in")

    def search(self, hashtag: str) -> None:
        driver.get(f"https://www.instagram.com/explore/tags/{hashtag}/")
        driver.implicitly_wait(3)

    def like(self, n: int) -> None:
        # generate indexes for pauses
        self.pauses = [
            55 * m + random.randint(1, 50) for m in range(1, int(n / 55))
        ]
        print("pausing at indexes: ", self.pauses)
        try:
            driver.find_element(By.CLASS_NAME, "_aagw").click()
        except ElementClickInterceptedException:
            print("Element click intercepted, post might already be opened")
        for i in range(n):
            try:
                if i in self.pauses:
                    sleep_time = random.randint(240, 800)
                    print(f"sleeping for {int(sleep_time / 60)} minutes")
                    time.sleep(sleep_time)
                # skip a random amount of posts
                print(f"post #{i+1}")
                if not random.randint(0, 9):
                    print("skipping this post")
                    continue
                svg_like = driver.find_element(
                    By.CSS_SELECTOR, "svg[aria-label='Like']"
                )
                # sleep for a random amount of time so bot doesn't look like a bot
                sleep_time = random.randint(2, 10)
                print(f"sleeping for {sleep_time} seconds")
                time.sleep(sleep_time)
                btn_like = svg_like.find_element(By.XPATH, "./../../..")
                btn_like.click()
                self.posts_liked += 1
                sleep_time = random.randint(1, 2)
                print(
                    f"liked post, sleeping for {sleep_time} seconds before moving onto next post"
                )
                time.sleep(sleep_time)
                driver.implicitly_wait(3)
                svg_next = driver.find_element(
                    By.CSS_SELECTOR, "svg[aria-label='Next']"
                )
                btn_next = svg_next.find_element(By.XPATH, "./../../..")
                btn_next.click()
                driver.implicitly_wait(3)
            except NoSuchElementException:
                print("No such element, checking if post is already liked:")
                svg_unlike = driver.find_element(
                    By.CSS_SELECTOR, "svg[aria-label='Unlike']"
                )

        print("posts liked: ", self.posts_liked)


ig = Instagram()

In [16]:
ig.login()

In [17]:
hashtags = [
    "creativewriting",
    "goodreads",
    "bookstagram",
    "bookworm",
    "poetry",
    "nonfiction",
    "fiction",
    "davidfosterwallace",
    "pulitzerprize",
    "bookclub",
    "bookreview",
    "penguinbooks",
    "writerslife",
    "memoir",
    "conceptualart",
    "writersofinstagram",
    "bookish",
    "flashfiction"
    "35mm",
]

ig.search("flashfiction")

In [20]:
mixer.init()
alarm = mixer.Sound("../driftveil.mp3")

try:
    ig.like(615)
except Exception as e:
    alarm.play()
    raise e

pausing at indexes:  [82, 111, 214, 224, 325, 344, 422, 483, 518, 587]


MaxRetryError: HTTPConnectionPool(host='localhost', port=60697): Max retries exceeded with url: /session/d88bc20062f86099b5ed4c35bb70710a/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x177ba8e80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [23]:
alarm.fadeout(1000)
driver.quit()